In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from textwrap import shorten
import requests

from bs4 import BeautifulSoup

In [2]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [10]:
df = pd.read_csv('New projects in Gowanus this year - Clean_ _ 50 units.csv')
df = df.head(16)

In [12]:
df['geo_address'] = df['Address'] + ' Gowanus, New York City'

In [13]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [14]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Image URL Collector

In [23]:
df['image_links'] = ''
links = []
for url in df['TRD Link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find('img',{'class':'MainArtImage_image__2YtGm'})
        images = images['src']
        links.append(images)
    except:
        links.append(None)

df['image_links'] = links

In [24]:
print(df['TRD Link'].iloc[2])

nan


In [25]:
print(df['image_links'].iloc[2])

None


In [41]:
# df.at[20,'image_img']=('<img src="Aimco.jpg" width="256" height="156">')
df['image_links'].iloc[1]

'https://static.therealdeal.com/wp-content/uploads/2023/06/main_MIA_Ella-Miami-Beach-1400x875.jpg'

In [42]:
# df.at[4,'image_links']=('https://therealdeal.com/wp-content/uploads/2019/10/trd-logo-16x9.jpg')

In [28]:
df["short_description"] = df["Owner/Developer"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['Owner/Developer'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["TRD Link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0     <a href="https://therealdeal.com/2021/10/20/qu...
1     <a href="https://therealdeal.com/2021/11/08/fi...
2                                                   NaN
3     <a href="https://therealdeal.com/2021/11/08/fi...
4                                                   NaN
5                                                   NaN
6     <a href="https://therealdeal.com/new-york/2023...
7                                                   NaN
8                                                   NaN
9                                                   NaN
10    <a href="https://therealdeal.com/new-york/2022...
11    <a href="https://therealdeal.com/new-york/2021...
12                                                  NaN
13                                                  NaN
14    <a href="https://therealdeal.com/new-york/2023...
15    <a href="https://therealdeal.com/new-york/2019...
Name: description_link, dtype: object

In [32]:
df['description_link']

0     <a href="https://therealdeal.com/2021/10/20/qu...
1     <a href="https://therealdeal.com/2021/11/08/fi...
2                                                   NaN
3     <a href="https://therealdeal.com/2021/11/08/fi...
4                                                   NaN
5                                                   NaN
6     <a href="https://therealdeal.com/new-york/2023...
7                                                   NaN
8                                                   NaN
9                                                   NaN
10    <a href="https://therealdeal.com/new-york/2022...
11    <a href="https://therealdeal.com/new-york/2021...
12                                                  NaN
13                                                  NaN
14    <a href="https://therealdeal.com/new-york/2023...
15    <a href="https://therealdeal.com/new-york/2019...
Name: description_link, dtype: object

In [53]:
def popup_html(row):
    i = row
    address = df['Address'].iloc[i]
    Owner_or_Developer = df['Owner/Developer'].iloc[i] if pd.isnull(df['description_link'].iloc[i]) else df['description_link'].iloc[i]
    total_const_floor_area = df['Total Construction Floor Area'].iloc[i]
    image = df['image_links'].iloc[i]
    
    image_html = '<img src="{}" width="256" height="156"><br>______________________________________<br>'.format(image) if pd.notnull(image) else ''
    
    html = '''<!DOCTYPE html>
    <html>''' + \
    image_html + \
    '''<strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Developer: </strong>{}'''.format(Owner_or_Developer) + '''<br>
    <strong>Total Construction Floor Area: </strong>{}'''.format(total_const_floor_area) + '''
    </html>
    '''
    return html

In [58]:
import branca

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=15)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Gowanus Projects')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html, width=256)
    popup = folium.Popup(folium.Html(html, script=True), max_width=256)
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [51]:
m.save('index.html')

## Map URL Snagger

In [52]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/NorthBeach_MiamiBeach
